In [11]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [12]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, BatchNormalization, Activation, Add, UpSampling2D
from tensorflow.keras.models import Model
import os
import matplotlib.pyplot as plt
from PIL import Image
from pathlib import Path


def preprocess_image(image_path):
    img = tf.io.read_file(image_path)
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.image.resize(img, [64, 64])  # Resize images to match HRNet input
    img /= 127.0  # Normalize to [0,1]
    return img

def load_dataset(image_dir):
    image_paths = [str(path) for path in Path(image_dir).glob("*.jpg")]
    dataset = tf.data.Dataset.from_tensor_slices(image_paths)
    dataset = dataset.map(preprocess_image)
    return dataset

# Load your dataset
train_dataset = load_dataset('/content/drive/MyDrive/dacl10k_v2_devphase/dacl10k_v2_devphase/images/train')
train_dataset = train_dataset.batch(32)  # Set the batch size


In [13]:
import numpy as np
import json
from PIL import Image, ImageDraw
import pandas as pd
import numpy as np
import keras

def create_mask_from_annotation(annot, img_size):
    mask = Image.new('L', img_size, 0)
    for shape in annot['shapes']:
        polygon = [(float(x), float(y)) for x, y in shape['points']]
        ImageDraw.Draw(mask).polygon(polygon, outline=1, fill=1)
    return np.array(mask)

def load_image_and_mask(img_path, annot_path, img_size=(64, 64)):
    img = Image.open(img_path)
    img = img.resize(img_size)
    img = np.array(img, dtype=np.float32) / 255.0  # Normalize and cast to float32

    with open(annot_path, 'r') as file:
        annot = json.load(file)
    mask = create_mask_from_annotation(annot, img_size)
    mask = np.array(mask, dtype=np.uint8)  # Cast mask to uint8
    mask = mask.reshape((*mask.shape, 1))  # Add channel dimension

    return img, mask

def calculate_iou(y_true, y_pred, smooth=1e-6):
    intersection = tf.reduce_sum(y_true * y_pred)
    union = tf.reduce_sum(y_true) + tf.reduce_sum(y_pred) - intersection
    iou = (intersection + smooth) / (union + smooth)
    return iou

def calculate_miou(y_true, y_pred, num_classes):
    iou_list = []
    for i in range(num_classes):
        y_true_i = tf.cast(y_true == i, tf.float32)
        y_pred_i = tf.cast(y_pred == i, tf.float32)
        iou_list.append(calculate_iou(y_true_i, y_pred_i))
    miou = tf.reduce_mean(iou_list)
    return miou

def segmentation_accuracy(y_true, y_pred):
    # Convert y_pred to label indices and reshape to match y_true shape
    y_pred_labels = tf.argmax(y_pred, axis=-1)
    y_pred_labels = tf.expand_dims(y_pred_labels, axis=-1)

    # Ensure y_true and y_pred_labels have the same shape
    y_true = tf.cast(y_true, tf.int64)
    y_pred_labels = tf.cast(y_pred_labels, tf.int64)

    # Calculate accuracy
    correct_prediction = tf.equal(y_pred_labels, y_true)
    return tf.reduce_mean(tf.cast(correct_prediction, tf.float32))


In [14]:
def load_dataset(image_dir, annotation_dir, batch_size=16):
    image_paths = sorted([str(path) for path in Path(image_dir).glob("*.jpg")])
    annotation_paths = sorted([str(path) for path in Path(annotation_dir).glob("*.json")])

    dataset = tf.data.Dataset.from_tensor_slices((image_paths, annotation_paths))
    dataset = dataset.map(lambda x, y: tf.numpy_function(
                            load_image_and_mask,
                            [x, y],
                            [tf.float32, tf.uint8]),  # Specify output types here
                          num_parallel_calls=tf.data.AUTOTUNE)
    dataset = dataset.map(lambda x, y: (tf.reshape(x, [64, 64, 3]), tf.reshape(y, [64, 64, 1])))
    dataset = dataset.batch(batch_size)
    return dataset
train_dataset = load_dataset('/content/drive/MyDrive/dacl10k_v2_devphase/dacl10k_v2_devphase/images/train', '/content/drive/MyDrive/dacl10k_v2_devphase/dacl10k_v2_devphase/annotations/train')
val_dataset = load_dataset('/content/drive/MyDrive/dacl10k_v2_devphase/dacl10k_v2_devphase/images/validation/', '/content/drive/MyDrive/dacl10k_v2_devphase/dacl10k_v2_devphase/annotations/validation/')
test_dataset = load_dataset('/content/drive/MyDrive/dacl10k_v2_devphase/dacl10k_v2_devphase/images/test/', '/content/drive/MyDrive/dacl10k_v2_devphase/dacl10k_v2_devphase/annotations/test/')

In [15]:
from tensorflow import keras

def get_optimized_alexnet_segmentation_model(input_shape=(32, 32, 3), num_classes=19):
    model = keras.models.Sequential([
        keras.layers.Conv2D(64, (3, 3), activation='relu', input_shape=input_shape, padding='same'),
        # Optional: MaxPooling layer can be adjusted or removed depending on the desired level of downsampling

        keras.layers.SeparableConv2D(128, (3, 3), activation='relu', padding='same'),
        # Optional: MaxPooling layer can be adjusted or removed

        keras.layers.SeparableConv2D(256, (3, 3), activation='relu', padding='same'),
        # Optional: MaxPooling layer can be adjusted or removed

        # Upsampling to 64x64
        keras.layers.UpSampling2D((2, 2)),  # This layer will upscale the feature map to 64x64
        keras.layers.Conv2D(128, (3, 3), activation='relu', padding='same'),

        # Final Convolutional Layer for Segmentation Mask
        keras.layers.Conv2D(num_classes, (3, 3), activation='softmax', padding='same')
    ])

    return model

# Instantiate the model
optimized_model = get_optimized_alexnet_segmentation_model()

# Compile the model
optimized_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Model summary
optimized_model.summary()


Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_9 (Conv2D)           (None, 32, 32, 64)        1792      
                                                                 
 separable_conv2d_6 (Separa  (None, 32, 32, 128)       8896      
 bleConv2D)                                                      
                                                                 
 separable_conv2d_7 (Separa  (None, 32, 32, 256)       34176     
 bleConv2D)                                                      
                                                                 
 up_sampling2d_3 (UpSamplin  (None, 64, 64, 256)       0         
 g2D)                                                            
                                                                 
 conv2d_10 (Conv2D)          (None, 64, 64, 128)       295040    
                                                      

In [ ]:

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow.keras import Model

# Assuming model, train_dataset, val_dataset, and test_dataset are already defined
# Assuming calculate_iou, calculate_miou, calculate_accuracy, calculate_loss are defined functions
# Assuming num_classes is the number of classes in your dataset

total_epochs = 90
interval_epochs = 30
metrics_history = []
num_classes = 19  # Replace with your actual number of classes

for interval in range(0, total_epochs, interval_epochs):
    # Train for interval_epochs epochs
    history = model.fit(train_dataset, epochs=interval_epochs, validation_data=val_dataset)

    # Function to calculate metrics for a dataset
    def evaluate_dataset(dataset, dataset_name):
        iou_sum = np.zeros(num_classes)
        miou_sum = np.zeros(num_classes)
        accuracy_sum, loss_sum, count = 0, 0, 0
        for x_batch, y_batch in dataset:
            y_pred = model.predict(x_batch)
            for y_true, y_pred_batch in zip(y_batch, y_pred):
                iou = calculate_iou(y_true, y_pred_batch)
                miou = calculate_miou(y_true, y_pred_batch)
                accuracy_sum += calculate_accuracy(y_true, y_pred_batch)
                loss_sum += calculate_loss(y_true, y_pred_batch)
                iou_sum += iou
                miou_sum += miou
                count += 1
        return iou_sum / count, miou_sum / count, accuracy_sum / count, loss_sum / count

    # Evaluate on validation set
    val_iou, val_miou, val_accuracy, val_loss = evaluate_dataset(val_dataset, "Validation")
    # Evaluate on test set
    test_iou, test_miou, test_accuracy, test_loss = evaluate_dataset(test_dataset, "Test")

    # Save the metrics
    metrics_history.append({
        'Epoch': interval + interval_epochs,
        'Validation IoU': val_iou,
        'Validation mIoU': val_miou,
        'Validation Accuracy': val_accuracy,
        'Validation Loss': val_loss,
        'Test IoU': test_iou,
        'Test mIoU': test_miou,
        'Test Accuracy': test_accuracy,
        'Test Loss': test_loss
    })

    # Save metrics to CSV
    metrics_df = pd.DataFrame(metrics_history)
    metrics_df.to_csv(f'training_metrics_up_to_epoch_{interval + interval_epochs}.csv', index=False)

# Plotting
# ... [Plotting code can be added here, modified to handle class-wise IoU and mIoU]

plt.tight_layout()
plt.show()


NameError: ignored